In [1]:
import json  
import numpy
import pandas as pd  
from pandas.io.json import json_normalize  
import requests
import io
import os


In [2]:
PATH=''
ENRICHMENT_TABLE_ID = '5d28fdeb7f099927ae2c2cd4'
BIG_PANDA_BEARER='4f1fb77d27cad48d23b189c4014414b3'


def import_json( j_file ):
    print ("importing data from {} ...".format(j_file))
    with open(os.path.join(PATH, j_file), encoding='utf-8') as f: 
        d = json.load(f) 
    
    df = json_normalize(d['records']) 
    print ('done. {}'.format(df.shape))
    return df


def generate_csv ( path, df, rows=0 ):   
    print('exporting csv...')
    if(rows<=0):
        df.to_csv(path,index=False)
    else:
        df.head(rows).to_csv(path,index=False)

    return

def send_data_bigpanda ( df, rows =0 ):
    
    print('exporting data to bigpanda...')

    s_buf = io.StringIO()
    generate_csv(s_buf, df)
    
    url = "https://api.bigpanda.io/resources/v1.0/enrichments/{}/map".format(ENRICHMENT_TABLE_ID)

    #print(url);

    headers = {
        'Authorization': "Bearer {}".format(BIG_PANDA_BEARER),
        'Content-Type': "text/csv;charset=utf8"
        }

    #print (headers)

    response = requests.request("POST", url, headers=headers, data=s_buf.getvalue().encode('utf-8'), stream=True)
    print('Result from bigpanda: {} \r\n{}'.format(response, response.text))

      





In [19]:
print('\r\n **** TIVIT AIOPS **** - Import CMDB to BIGPANDA\r\n\r\n')

print('starting importing jsons...')
#load jsons to pandas dataframe  
df_so = import_json('cmdb_server_bigpanda.json')
df_rel = import_json('cmdb_relacionamento_bigpanda.json')
df_cmdb = import_json('cmdb_bigpanda.json')


print('\r\nsanitizing cmdb fields...')
#remover | dos names pois são usados como array no bigpanda.
df_cmdb['name'] = df_cmdb['name'].str.replace('|','-').str.replace('\n',' ')
df_cmdb['fqdn'] = df_cmdb['fqdn'].str.replace('|','-').str.replace('\n',' ')
df_cmdb['ip_address'] = df_cmdb['ip_address'].str.replace('\n',' ')


print('extracting dataframes relationships...')

print('extracting cluster information from cmdb...')
df_clusters= df_rel.merge(df_cmdb[(df_cmdb.sys_class_name=='cmdb_ci_cluster')], left_on='parent', right_on='sys_id', how='inner')[['parent', 'sys_id','child', 'name', 'company.name']]

print('extracting businessapp from cmdb...')
df_business_app= df_rel.merge(df_cmdb[(df_cmdb.sys_class_name=='cmdb_ci_business_app')], left_on='parent', right_on='sys_id', how='inner')[['parent', 'sys_id','child', 'name', 'company.name']]

print('extractig services from cmdb...')
df_service= df_rel.merge(df_cmdb[(df_cmdb.sys_class_name=='cmdb_ci_service')], left_on='parent', right_on='sys_id', how='inner')[['parent', 'sys_id','child', 'name', 'company.name']]

print('extracting aplication from cmdb...')
df_application= df_rel.merge(df_cmdb[df_cmdb.sys_class_name.isin(['cmdb_ci_application_software','cmdb_ci_appl'])], left_on='parent', right_on='sys_id', how='inner')[['parent', 'sys_id','child', 'name', 'company.name']]

print('extracting operating systems from cmdb...')
df_os= df_so[['name', 'fqdn', 'os', 'os_version']].drop_duplicates()    

print('create relationships with cmdb values(fqdn, sys_id)')
df_rel2=df_rel.merge(df_cmdb, left_on='parent', right_on='sys_id').merge(df_cmdb, left_on='child', right_on='sys_id')[['child', 'parent', 'name_x', 'sys_class_name_x', 'name_y','sys_class_name_y']].rename(columns={
'child':'child',
'parent':'parent',
'name_x':'parent_name',
'sys_class_name_x':'parent_sys_class_name',
'name_y':'child_name',
'sys_class_name_y':'child_sys_class_name'})



print('creating grouping and merging dataframes...')
#dados cluster
df1= df_cmdb.merge(df_clusters, left_on='sys_id',right_on='child', how='inner')[['sys_id_x','name_x', 'name_y']].groupby(['sys_id_x','name_x'])['name_y'].apply(list).to_frame().reset_index().rename(columns={'sys_id_x':'sys_id','name_x':'name_x','name_y':'parents'})  
#dados childs
df2= df_cmdb.merge(df_rel2, left_on='sys_id',right_on='parent', how='inner')[['sys_id','name', 'child_name']].groupby(['sys_id','name'])['child_name'].apply(list).to_frame().reset_index().rename(columns={'sys_id':'sys_id','name':'name_x','child_name':'childs'})  
#dados parents
df3= df_cmdb.merge(df_rel2, left_on='sys_id',right_on='child', how='inner')[['sys_id','name', 'parent_name']].groupby(['sys_id','name'])['parent_name'].apply(list).to_frame().reset_index().rename(columns={'sys_id':'sys_id','name':'name_x','parent_name':'clusters'}) 
#dados os
df4= df_cmdb.merge(df_os, left_on=['name','fqdn'],right_on=['name','fqdn'], how='inner')[['sys_id','name', 'os']].groupby(['sys_id','name'])['os'].apply(list).to_frame().reset_index().rename(columns={'sys_id':'sys_id','name':'name_x','os':'so'}) 
#dados application
df5= df_cmdb.merge(df_application, left_on='sys_id',right_on='child', how='inner')[['sys_id_x','name_x', 'name_y']].groupby(['sys_id_x','name_x'])['name_y'].apply(list).to_frame().reset_index().rename(columns={'sys_id_x':'sys_id','name_x':'name_x','name_y':'apps'})
#dados services
df6= df_cmdb.merge(df_service, left_on='sys_id',right_on='child', how='inner')[['sys_id_x','name_x', 'name_y']].groupby(['sys_id_x','name_x'])['name_y'].apply(list).to_frame().reset_index().rename(columns={'sys_id_x':'sys_id','name_x':'name_x','name_y':'services'})
#dados business application
df7= df_cmdb.merge(df_business_app, left_on='sys_id',right_on='child', how='inner')[['sys_id_x','name_x', 'name_y']].groupby(['sys_id_x','name_x'])['name_y'].apply(list).to_frame().reset_index().rename(columns={'sys_id_x':'sys_id','name_x':'name_x','name_y':'business_app'})
print('done')

# print ('clusters',df_clusters.shape)
# print ('df_cmdb',df_cmdb.shape)
# print ('df_application',df_application.shape)
# print ('df_business_app',df_business_app.shape)
# print ('df_service',df_service.shape)
# print ('df_os',df_os.shape)
# print ('df_rel2',df_rel2.shape)  


print('\r\ngenerating final dataframe ...')
#generating final dataframe
df_final=df_cmdb.merge(df1, left_on='sys_id',right_on='sys_id', how='left').merge(df2, left_on='sys_id',right_on='sys_id', how='left').merge(df3, left_on='sys_id',right_on='sys_id', how='left').merge(df4, left_on='sys_id',right_on='sys_id', how='left').merge(df5, left_on='sys_id',right_on='sys_id', how='left').merge(df6, left_on='sys_id',right_on='sys_id', how='left').merge(df7, left_on='sys_id',right_on='sys_id', how='left')[['sys_id', 'sys_class_name', 'name', 'fqdn', 'ip_address','location.name', 'company.name', 'parents', 'childs', 'clusters', 'so', 'apps', 'services', 'business_app', 'u_impact' ]].rename(columns={'location.name':'location', 'company.name':'customer', 'ip_address':'ip', 'sys_class_name':'category', 'parents':'dependencies_parent', 'childs':'dependencies_child', 'apps':'application', 'business_app':'business_application', 'clusters':'cluster','so':'os', 'u_impact':'impact', 'fqdn':'fqdn' })

#change arrays to string concatenated with |
df_final['cluster'] = df_final['cluster'].str.join('|')
df_final['dependencies_parent'] = df_final['dependencies_parent'].str.join('|')
df_final['dependencies_child'] = df_final['dependencies_child'].str.join('|')
df_final['os'] = df_final['os'].str.join('|')
df_final['application'] = df_final['application'].str.join('|')
df_final['services'] = df_final['services'].str.join('|')
df_final['business_application'] = df_final['business_application'].str.join('|')

#reordening columns
df_final = df_final[['sys_id','category','name','fqdn','customer','ip','os','location','impact','dependencies_parent','dependencies_child','cluster','application','business_application','services']]

#send_data_bigpanda(df_final, 1000)

#generate_csv('cmdb.csv',df_final)






 **** TIVIT AIOPS **** - Import CMDB to BIGPANDA


starting importing jsons...
importing data from cmdb_server_bigpanda.json ...
done. (122420, 9)
importing data from cmdb_relacionamento_bigpanda.json ...
done. (109270, 7)
importing data from cmdb_bigpanda.json ...
done. (333033, 21)

sanitizing cmdb fields...
extracting dataframes relationships...
extracting cluster information from cmdb...
extracting businessapp from cmdb...
extractig services from cmdb...
extracting aplication from cmdb...
extracting operating systems from cmdb...
create relationships with cmdb values(fqdn, sys_id)
creating grouping and merging dataframes...
done

generating final dataframe ...


In [5]:
df_cmdb[df_cmdb.name=='WSPSQLBRZ025'].head()

,__status,company.name,dv_company.name,dv_fqdn,dv_install_status,dv_ip_address,dv_location.name,dv_name,dv_short_description,dv_sys_class_name,...,dv_u_impact,fqdn,install_status,ip_address,location.name,name,short_description,sys_class_name,sys_id,u_impact
162360,success,ZURICH SEGUROS S/A,ZURICH SEGUROS S/A,NDM Removido em: 29/11/2016 as 13:55:02,None,172.26.3.73,TIVIT - TRANSAMERICA,WSPSQLBRZ025,,Servidor,...,1,NDM Removido em: 29/11/2016 as 13:55:02,0,172.26.3.73,TIVIT - TRANSAMERICA,WSPSQLBRZ025,,cmdb_ci_server,8c31c2dddb52bf84817ea19b8a961980,1


In [27]:
df_final[~pd.isnull(df_final.services)][['name','fqdn', 'category', 'services']]
df_final[~pd.isnull(df_final.business_application)][['name','fqdn', 'category', 'business_application']]

,name,fqdn,category,business_application
17,TSC2 - PROD,,cmdb_ci_appl,TSC2
66,CONTROL-M V7 DR,,cmdb_ci_appl,CONTROL-M V7
195,TSC2 - DEV,,cmdb_ci_appl,TSC2
437,CONTROL-M V7 PROD,,cmdb_ci_appl,CONTROL-M V7
721,TSC2 - ADMIN,,cmdb_ci_appl,TSC2
732,ANALYTICS - BANCO DE DADOS,,cmdb_ci_appl,ANALYTICS
766,CONTROL-M V7 HOMOLOG,,cmdb_ci_appl,CONTROL-M V7
767,TSC2 - DEV,,cmdb_ci_appl,TSC2
105107,SRVSVWSP15,FNIS-SRVSVWSP15.WIN.SP1,cmdb_ci_server,FIDELITY_PR_DI_DOWN_TB_SVIEW_VALID_SRVSVWSP15|...
105313,LNXORASP11,FNIS-LNXORASP11.LNX.SP1,cmdb_ci_server,FIDELITY_BCV_END_BACKUP_LNXORASP|FIDELITY_BCV_...
